In [ ]:
count features, cumcount features, time-delta features, unique-count features, and which [app/os/channel]s each IP appears in the data.

In [ ]:
As we all know, the click delta is important,so I fed deltas of last 5 and next 5 click_times to the network and designed a model with RNN cell to find the patterns of the click series

In [17]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt
import pickle
from sklearn.metrics import roc_auc_score
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

alt.renderers.enable('notebook')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


RendererRegistry.enable('notebook')

In [4]:
main_path = r'../..'

import sys
sys.path.append(main_path)
from BayDS import *
# from lib.training import plot_importance

experiment_name = '31.08'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [48]:
X.index

Int64Index([2987000, 2987001, 2987002, 2987003, 2987004, 2987005, 2987006,
            2987007, 2987008, 2987009,
            ...
            3577530, 3577531, 3577532, 3577533, 3577534, 3577535, 3577536,
            3577537, 3577538, 3577539],
           dtype='int64', name='TransactionID', length=590540)

In [9]:
train = pd.read_pickle(f'{main_learning_folder}/train.pkl')
test = pd.read_pickle(f'{main_learning_folder}/test.pkl')


X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT'], axis=1)
y = train.sort_values('TransactionDT')['isFraud'].astype(np.uint8)
test = test.sort_values('TransactionDT').drop(['TransactionDT'], axis=1)

X.drop('Date', axis=1, inplace=True)
X=X.astype(np.float32)
test.drop('Date', axis=1, inplace=True)
test=test.astype(np.float32)
del train
gc.collect()

In [13]:
paramstest = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }

In [34]:
def est(Xtest):
    lgbtest = lgb.LGBMClassifier(**paramstest)
    lgbtest.fit(Xtest, y)

    print(round(roc_auc_score(y, lgbtest.predict_proba(Xtest)[:, 1])*100, 2))

In [ ]:
Xtest = X[['TransactionAmt']].copy()
Xtest['dec'] = ((X['TransactionAmt'] - X['TransactionAmt'].astype(int)) * 1000).astype(int)
Xtest['dec1'] = ((X['TransactionAmt']+0.5 - (X['TransactionAmt']+0.5).astype(int)) * 1000).astype(int)

In [45]:
Xtest['tr'] = np.log(Xtest['TransactionAmt'])
Xtest['dec1'] = ((Xtest['tr'] - (Xtest['tr']).astype(int)) * 1000).astype(int)

In [46]:
est(Xtest[['dec1']])

69.91


In [36]:
est(Xtest[['dec']])

68.48


In [38]:
est(Xtest[['TransactionAmt']])

71.47


In [41]:
space_lgb = {
    'num_leaves': np.arange(10, 100, 30),
    'min_child_weight': np.arange(1, 300, 80),
    'learning_rate': [np.float32(0.01)],
    'max_bin': np.arange(9, 40, 1),
    'lambda_l1': np.arange(0, 3, 1), 
    'lambda_l2': np.arange(0, 3, 1),
    'min_data_in_leaf' : np.arange(5, 250, 90),
    'min_sum_hessian_in_leaf' : np.arange(0.0001, 0.01, 0.005),
    'bagging_fraction' : np.arange(0.1, 0.9, 0.3),
    'max_bin': np.arange(10, 30, 10),
    'feature_fraction' : np.arange(0.1, 0.9, 0.3),
    'bagging_freq' : np.arange(200, 1000, 400),
    'min_gain_to_split': np.arange(0.1, 1, 0.3),     
}
import itertools
result = list(itertools.product(*space_lgb.values()))
import random
random.seed(77)
random.shuffle(result)

In [ ]:
num_experiments = len(result)
for it,r in enumerate(result):
    if it == 0:
        continue
    gc.collect()
    print(f'######################\n{it}/{num_experiments}')
    tuned_params_lgb = {k:r[i].item() for i,k in enumerate(space_lgb.keys())}
    print(tuned_params_lgb)
    # Setting model_folder
    model_name = 'lightgbm-%04d'%it
    model_folder = f'{main_learning_folder}/{model_name}'
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    n_fold = 5
    # folds = TimeSeriesSplit(n_splits=n_fold)
    folds = KFold(n_splits=n_fold)
    # folds = GroupKFold(n_splits=5)
    # groups = pd.read_pickle('./groups.pkl').values
    
    params = { **tuned_params_lgb,
          'objective': 'binary',
          'max_depth': -1,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47,
#           'device': 'gpu'
         }
    train_options = {
        "model_type":'lgb',
        "params": params,
        "eval_metric":'auc',
        'early_stopping_rounds': 500,
        'n_estimators': 10000,
        'averaging': 'usual',
        'use_groups': False,
        'fold_name': folds.__class__.__name__,
        'n_splits': n_fold
    }
    with open(f'{model_folder}/training_params.json', 'w') as f:
        q = json.dumps(train_options,indent=2)
        f.write(q)
    
    result_dict_lgb = train_model_classification(X=X, X_test=test, y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             verbose=500, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
                                             n_jobs=-1, groups=None)
    # plt.savefig(f'{learning_folder}/feature_importance.png')

    plot_importance(result_dict_lgb,f'{model_folder}/feature_importance.png')
    
    with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
    #     q = json.dumps(result_dict_lgb,indent=2)
        pickle.dump(result_dict_lgb,f)
    #     f.write(q)

######################
1/69984
{'num_leaves': 70, 'min_child_weight': 161, 'learning_rate': 0.009999999776482582, 'max_bin': 10, 'lambda_l1': 1, 'lambda_l2': 2, 'min_data_in_leaf': 95, 'min_sum_hessian_in_leaf': 0.0051, 'bagging_fraction': 0.7000000000000001, 'feature_fraction': 0.7000000000000001, 'bagging_freq': 200, 'min_gain_to_split': 0.7000000000000001}
Fold 1 started at Sun Sep  1 14:24:40 2019
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.946633	training's auc: 0.946633	valid_1's auc: 0.892312	valid_1's auc: 0.892312
[1000]	training's auc: 0.967352	training's auc: 0.967352	valid_1's auc: 0.908999	valid_1's auc: 0.908999
[1500]	training's auc: 0.976816	training's auc: 0.976816	valid_1's auc: 0.913979	valid_1's auc: 0.913979
[2000]	training's auc: 0.981869	training's auc: 0.981869	valid_1's auc: 0.916295	valid_1's auc: 0.916295
[2500]	training's auc: 0.985457	training's auc: 0.985457	valid_1's auc: 0.917064	valid_1's auc: 0.917064
[3000]	t

[4500]	training's auc: 0.937053	training's auc: 0.937053	valid_1's auc: 0.905116	valid_1's auc: 0.905116
[5000]	training's auc: 0.938991	training's auc: 0.938991	valid_1's auc: 0.90552	valid_1's auc: 0.90552
[5500]	training's auc: 0.941275	training's auc: 0.941275	valid_1's auc: 0.906822	valid_1's auc: 0.906822
[6000]	training's auc: 0.943095	training's auc: 0.943095	valid_1's auc: 0.90754	valid_1's auc: 0.90754
[6500]	training's auc: 0.945714	training's auc: 0.945714	valid_1's auc: 0.909495	valid_1's auc: 0.909495
[7000]	training's auc: 0.946665	training's auc: 0.946665	valid_1's auc: 0.910399	valid_1's auc: 0.910399
[7500]	training's auc: 0.948882	training's auc: 0.948882	valid_1's auc: 0.911088	valid_1's auc: 0.911088
[8000]	training's auc: 0.950915	training's auc: 0.950915	valid_1's auc: 0.911506	valid_1's auc: 0.911506
Early stopping, best iteration is:
[7864]	training's auc: 0.950521	training's auc: 0.950521	valid_1's auc: 0.911609	valid_1's auc: 0.911609
Fold 4 started at Sun Se

[7500]	training's auc: 0.961512	training's auc: 0.961512	valid_1's auc: 0.914265	valid_1's auc: 0.914265
[8000]	training's auc: 0.963048	training's auc: 0.963048	valid_1's auc: 0.914393	valid_1's auc: 0.914393
[8500]	training's auc: 0.964609	training's auc: 0.964609	valid_1's auc: 0.914977	valid_1's auc: 0.914977
[9000]	training's auc: 0.966005	training's auc: 0.966005	valid_1's auc: 0.915607	valid_1's auc: 0.915607
[9500]	training's auc: 0.967263	training's auc: 0.967263	valid_1's auc: 0.916869	valid_1's auc: 0.916869
[10000]	training's auc: 0.968443	training's auc: 0.968443	valid_1's auc: 0.917431	valid_1's auc: 0.917431
Did not meet early stopping. Best iteration is:
[10000]	training's auc: 0.968443	training's auc: 0.968443	valid_1's auc: 0.917431	valid_1's auc: 0.917431
CV mean score: 0.9222, std: 0.0089.
######################
5/69984
{'num_leaves': 70, 'min_child_weight': 1, 'learning_rate': 0.009999999776482582, 'max_bin': 20, 'lambda_l1': 0, 'lambda_l2': 2, 'min_data_in_leaf': 

[10000]	training's auc: 0.960072	training's auc: 0.960072	valid_1's auc: 0.934811	valid_1's auc: 0.934811
Did not meet early stopping. Best iteration is:
[10000]	training's auc: 0.960072	training's auc: 0.960072	valid_1's auc: 0.934811	valid_1's auc: 0.934811
Fold 5 started at Mon Sep  2 10:59:29 2019
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.892947	training's auc: 0.892947	valid_1's auc: 0.870765	valid_1's auc: 0.870765
[1000]	training's auc: 0.907133	training's auc: 0.907133	valid_1's auc: 0.883224	valid_1's auc: 0.883224
[1500]	training's auc: 0.914973	training's auc: 0.914973	valid_1's auc: 0.888492	valid_1's auc: 0.888492
[2000]	training's auc: 0.921401	training's auc: 0.921401	valid_1's auc: 0.894231	valid_1's auc: 0.894231
[2500]	training's auc: 0.926518	training's auc: 0.926518	valid_1's auc: 0.896694	valid_1's auc: 0.896694
[3000]	training's auc: 0.93112	training's auc: 0.93112	valid_1's auc: 0.899668	valid_1's auc: 0.899668
[3500]	

{'num_leaves': 10, 'min_child_weight': 241, 'learning_rate': 0.009999999776482582, 'max_bin': 10, 'lambda_l1': 2, 'lambda_l2': 0, 'min_data_in_leaf': 95, 'min_sum_hessian_in_leaf': 0.0001, 'bagging_fraction': 0.4, 'feature_fraction': 0.7000000000000001, 'bagging_freq': 600, 'min_gain_to_split': 0.1}
Fold 1 started at Tue Sep  3 00:27:45 2019
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.896012	training's auc: 0.896012	valid_1's auc: 0.861088	valid_1's auc: 0.861088
[1000]	training's auc: 0.909931	training's auc: 0.909931	valid_1's auc: 0.876889	valid_1's auc: 0.876889
[1500]	training's auc: 0.917953	training's auc: 0.917953	valid_1's auc: 0.882725	valid_1's auc: 0.882725
[2000]	training's auc: 0.924785	training's auc: 0.924785	valid_1's auc: 0.88859	valid_1's auc: 0.88859
[2500]	training's auc: 0.930495	training's auc: 0.930495	valid_1's auc: 0.893784	valid_1's auc: 0.893784
[3000]	training's auc: 0.936901	training's auc: 0.936901	valid_1's auc:

{'num_leaves': 70, 'min_child_weight': 1, 'learning_rate': 0.009999999776482582, 'max_bin': 10, 'lambda_l1': 1, 'lambda_l2': 2, 'min_data_in_leaf': 5, 'min_sum_hessian_in_leaf': 0.0001, 'bagging_fraction': 0.7000000000000001, 'feature_fraction': 0.7000000000000001, 'bagging_freq': 600, 'min_gain_to_split': 0.4}
Fold 1 started at Tue Sep  3 10:21:30 2019
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.945836	training's auc: 0.945836	valid_1's auc: 0.893087	valid_1's auc: 0.893087
[1000]	training's auc: 0.965644	training's auc: 0.965644	valid_1's auc: 0.907653	valid_1's auc: 0.907653
[1500]	training's auc: 0.97596	training's auc: 0.97596	valid_1's auc: 0.912887	valid_1's auc: 0.912887
[2000]	training's auc: 0.981943	training's auc: 0.981943	valid_1's auc: 0.915279	valid_1's auc: 0.915279
[2500]	training's auc: 0.985798	training's auc: 0.985798	valid_1's auc: 0.91711	valid_1's auc: 0.91711
[3000]	training's auc: 0.988909	training's auc: 0.988909	vali

[7500]	training's auc: 0.943104	training's auc: 0.943104	valid_1's auc: 0.893233	valid_1's auc: 0.893233
[8000]	training's auc: 0.944417	training's auc: 0.944417	valid_1's auc: 0.891755	valid_1's auc: 0.891755
Early stopping, best iteration is:
[7797]	training's auc: 0.943643	training's auc: 0.943643	valid_1's auc: 0.893467	valid_1's auc: 0.893467
Fold 2 started at Wed Sep  4 04:03:06 2019
Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.887813	training's auc: 0.887813	valid_1's auc: 0.881905	valid_1's auc: 0.881905
[1000]	training's auc: 0.899703	training's auc: 0.899703	valid_1's auc: 0.888471	valid_1's auc: 0.888471
[1500]	training's auc: 0.90681	training's auc: 0.90681	valid_1's auc: 0.893023	valid_1's auc: 0.893023
[2000]	training's auc: 0.912569	training's auc: 0.912569	valid_1's auc: 0.895658	valid_1's auc: 0.895658
[2500]	training's auc: 0.917133	training's auc: 0.917133	valid_1's auc: 0.897305	valid_1's auc: 0.897305
[3000]	training's auc: